In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import linear_model, tree, ensemble
import umap

In [13]:
dataset = pd.read_csv("data_tg_dupl_rem.csv")

In [14]:
dataset

,Li,Be,B,O,Na,Mg,Al,Si,P,K,...,Hf,Ta,W,Hg,Tl,Pb,Bi,Th,U,Tg
0,0.000000,0.0,0.0,0.631579,0.000000,0.000000,0.035088,0.245614,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,951.15
1,0.000000,0.0,0.0,0.627119,0.000000,0.000000,0.067797,0.220339,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,966.15
2,0.000000,0.0,0.0,0.622951,0.000000,0.000000,0.098361,0.196721,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,983.15
3,0.000000,0.0,0.0,0.619048,0.000000,0.000000,0.126984,0.174603,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1012.15
4,0.000000,0.0,0.0,0.589063,0.134375,0.000000,0.062500,0.214062,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,898.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43636,0.000000,0.0,0.0,0.666667,0.000000,0.055556,0.000000,0.000000,0.222222,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,798.65
43637,0.125000,0.0,0.0,0.625000,0.000000,0.000000,0.000000,0.000000,0.208333,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,610.65
43638,0.190476,0.0,0.0,0.619048,0.000000,0.000000,0.000000,0.000000,0.095238,0.0,...,0.0,0.0,0.095238,0.0,0.0,0.0,0.0,0.0,0.0,709.15
43639,0.000000,0.0,0.0,0.600000,0.000000,0.000000,0.000000,0.266667,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,767.15


# Pré-processamento dos dados

In [15]:
#remoção da coluna 'Tg' (alvo)
dados_dim65 = dataset.drop(['Tg'], axis=1)

# Min max scaler

In [16]:
# Aplicação Min Max Scaler para todas as 65 colunas dos dados
# cada elemento terá a mesma contribuição para o agrupamento dos dados
scaler = MinMaxScaler(copy=False)
scaler.fit_transform(dados_dim65)
print(dados_dim65)

             Li   Be    B         O        Na        Mg        Al        Si  \
0      0.000000  0.0  0.0  0.735648  0.000000  0.000000  0.095723  0.736842   
1      0.000000  0.0  0.0  0.725257  0.000000  0.000000  0.184957  0.661017   
2      0.000000  0.0  0.0  0.715548  0.000000  0.000000  0.268339  0.590164   
3      0.000000  0.0  0.0  0.706455  0.000000  0.000000  0.346427  0.523810   
4      0.000000  0.0  0.0  0.636604  0.236871  0.000000  0.170507  0.642188   
...         ...  ...  ...       ...       ...       ...       ...       ...   
43636  0.000000  0.0  0.0  0.817386  0.000000  0.243986  0.000000  0.000000   
43637  0.213937  0.0  0.0  0.720322  0.000000  0.000000  0.000000  0.000000   
43638  0.325999  0.0  0.0  0.706455  0.000000  0.000000  0.000000  0.000000   
43639  0.000000  0.0  0.0  0.662083  0.000000  0.000000  0.000000  0.800000   
43640  0.000000  0.0  0.0  0.685379  0.199779  0.000000  0.000000  0.830000   

              P    K  ...   Lu   Hf   Ta         W 

In [23]:
X = dados_dim65.iloc[:,:].values
y = dataset.iloc[:,-1].values
print("Matrix of features", X, sep='\n')
print("--------------------------------------------------")
print("Target Variable", y, sep='\n')
print(X.shape)
print(y.shape)

Matrix of features
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.32599942 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
--------------------------------------------------
Target Variable
[951.15 966.15 983.15 ... 709.15 767.15 768.15]
(43641, 64)
(43641,)


In [24]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)
kf.get_n_splits(X)

10

In [9]:
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f" Train: index={train_index}")
    print(f" Test: index={test_index}")

Fold 0:
 Train: index=[    0     1     2 ... 43638 43639 43640]
 Test: index=[   10    40    44 ... 43594 43607 43626]
Fold 1:
 Train: index=[    0     1     2 ... 43638 43639 43640]
 Test: index=[    3     8    23 ... 43596 43598 43599]
Fold 2:
 Train: index=[    0     1     3 ... 43638 43639 43640]
 Test: index=[    2    11    19 ... 43628 43630 43636]
Fold 3:
 Train: index=[    0     1     2 ... 43638 43639 43640]
 Test: index=[   21    26    36 ... 43609 43616 43622]
Fold 4:
 Train: index=[    1     2     3 ... 43638 43639 43640]
 Test: index=[    0    13    29 ... 43615 43619 43620]
Fold 5:
 Train: index=[    0     1     2 ... 43638 43639 43640]
 Test: index=[    4     6     7 ... 43624 43632 43634]
Fold 6:
 Train: index=[    0     1     2 ... 43638 43639 43640]
 Test: index=[    5    12    49 ... 43610 43631 43635]
Fold 7:
 Train: index=[    0     2     3 ... 43636 43639 43640]
 Test: index=[    1    75    80 ... 43618 43637 43638]
Fold 8:
 Train: index=[    0     1     2 ... 436

In [10]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

In [11]:
score = cross_val_score(ensemble.RandomForestRegressor(random_state=42), X, y, cv= kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold are: {score}')
rmse(score.mean())

Scores for each fold are: [ -875.8164052  -1041.3932598  -1056.79115267 -1074.96867781
 -1009.71153309 -1139.4533868   -964.70751357 -1065.98774135
 -1167.76412181  -962.60211235]
rmse= 32.19
